In [1]:
import jieba
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
import pickle
from sklearn.externals import joblib

## todo 效果可能不好的原因：
* 特征没有提取好：分词部分、停词的问题?字典的问题?卡方检验的参数设置?
* 模型本身的问题（vsm的锅？）
* 数据量是否足够?
* tf-idf的问题?嗯..似乎频次都是1,暂时没有tf-idf需求吧?


In [2]:
# 生成vsm词袋模型
bag_of_words ={}
text_list =[]
_label = []

search_pd = pd.read_csv('./search.csv' ,header= -1)
# todo 技术方案不好的原因...?
for s in search_pd.iterrows():
    seg_list = jieba.lcut(s[1][0], cut_all=False)
    text_list.append(seg_list)
    _label.append(0)
    for seg in seg_list:
        if seg not in bag_of_words:
            bag_of_words[seg] = 1
        else:
            bag_of_words[seg] += 1

pub_pd = pd.read_csv('./pub.csv' ,header= -1)
for s in pub_pd.iterrows():
    seg_list = jieba.lcut(s[1][0], cut_all=False)
    text_list.append(seg_list)
    _label.append(1)
    for seg in seg_list:
        if seg not in bag_of_words:
            bag_of_words[seg] = 1
        else:
            bag_of_words[seg] += 1



Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\59256\AppData\Local\Temp\jieba.cache
Loading model cost 0.677 seconds.
Prefix dict has been built succesfully.


In [3]:
vsm_vec = {}
row_vsm_vec = []
count = 0
for key,value in bag_of_words.items():
    sub_dic = {}
    
    # 其他的信息,比如重要性,tf-idf值之类的信息也保存再这里
    sub_dic['index'] = count
    count += 1
    vsm_vec[key] = sub_dic
    
    row_vsm_vec.append(key)
    
    sub_dic['count'] = value
    # todo 增加 tf-idf的值!!!

In [4]:
# 生成向量
vec_col_count = len(vsm_vec)
_list = []

for l in text_list:
    self_bag = {}
    row_vec = [0]*vec_col_count
    for wd in l:
        if wd in self_bag:
            self_bag[wd] += 1
        else:
            self_bag[wd] = 1
    for b,c in self_bag.items():
        if b in vsm_vec:
            row_vec[vsm_vec[b]['index']] = c
    _list.append(row_vec)

In [5]:
# 卡方检验得到有效特征
chi2 = SelectKBest(chi2,k=60).fit(_list, _label)

In [6]:
# 输出rank的score！！
top = 100

f_ip = {}
for col_index in range(0,vec_col_count):
    f_ip[col_index] = chi2.scores_[col_index]

items = f_ip.items()
it = sorted(items,key=lambda d:d[1], reverse = True)
f_ip_array = []

for key,value in it:
    f_ip_array.append(key)

f_ip_array = f_ip_array[0:top]

In [7]:
chi2_vsm_vec = {}

chi2_count = 0
for col_index in f_ip_array:
    dic = vsm_vec[row_vsm_vec[col_index]]
    dic['index'] = chi2_count
    chi2_count += 1
    chi2_vsm_vec[row_vsm_vec[col_index]] = dic



In [8]:
# 生成向量
chi2_vec_col_count = len(chi2_vsm_vec)
_chi2_list = []


for l in text_list:
    self_bag = {}
    row_vec = [0]*chi2_vec_col_count
    for wd in l:
        if wd in self_bag:
            self_bag[wd] += 1
        else:
            self_bag[wd] = 1
    for b,c in self_bag.items():
        if b in chi2_vsm_vec:
            row_vec[chi2_vsm_vec[b]['index']] = c
    _chi2_list.append(row_vec)


In [9]:
clf = RandomForestClassifier(n_estimators=40,oob_score= True)
clf = clf.fit(_chi2_list, _label)

# 用随机森林做~ 嗯...为什么呢...
clf.oob_score_

0.97169811320754718

In [10]:
# 保存相关参数~
chi2_vsm_vec
with open('./chi2_vsm_vec','wb') as f:
    pickle.dump(chi2_vsm_vec,f)
joblib.dump(clf, "train_model.m")

['train_model.m']

In [12]:
# 预测流程~

txt = ['找一下水杯的图片','发个话题吧!','我要发一个话题','来一张长城的图片','来张壁纸','百度搜图','hi! 我要搜图']

# 加载模型~
with open('./chi2_vsm_vec','rb') as f:
    n_chi2_vsm_vec = pickle.load(f)
rf_clf = joblib.load('./train_model.m')


for t in txt:
    seg_list = jieba.lcut(t, cut_all=False)

    # 生成向量
    chi2_vec_col_count = len(n_chi2_vsm_vec)
    _chi2_list = []

    l = seg_list

    self_bag = {}
    row_vec = [0]*chi2_vec_col_count
    for wd in l:
        if wd in self_bag:
            self_bag[wd] += 1
        else:
            self_bag[wd] = 1
    for b,c in self_bag.items():
        if b in n_chi2_vsm_vec:
            row_vec[n_chi2_vsm_vec[b]['index']] = c

    print(rf_clf.predict([row_vec]))


[0]
[1]
[1]
[0]
[1]
[1]
[1]
